### Installs and imports

In [ ]:
!pip3 install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
import tweepy
import ssl
import time
import pandas as pd

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context

### Declare authentication keys

In [ ]:
# Twitter authentication keys
consumer_key = " "
consumer_secret = " "
access_token = " "
access_token_secret = " "

In [ ]:
# Authentication with Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Scrape tweets from a specified user

In [41]:
my_tweets = api.user_timeline(id='larrouturou', count=15)
for tweet in my_tweets:
  print (tweet.text)

RT @Fridays4FutureU: This morning floods have killed at least three people and several others missing in eastern Uganda. Rivers Nabuyonga a…
RT @Sudhirntv: A disaster that Mbale people never saw coming. 

Three people are said to have so far died following a heavy downpour which…
RT @Sudhirntv: #MbaleDisaster https://t.co/i7spwzRDrS
Ce qui se passe “équivaut à des incendies sous-marins, une faune et flore qui meurent comme si elles étaient brûlée… https://t.co/n2GQdYtc5F
RT @lucas_chancel: En ce jour du dépassement planétaire, il faut rappeler que la moitié de tout ce qui a été consommé depuis janvier (en to…
RT @audreygarric: La sécheresse est bien installée sur la France : 90 départements sur 96 font l’objet, depuis hier, de restrictions d’usag…
RT @ZLabe: 2016, 2019, 2020, 2021, and 2022 are the five hottest months of June on record for the Northern Hemisphere 🤔

[Data information:…
RT @audreygarric: C'est sans fin. Le violent incendie « Oak fire » continue de s’étendre en Californi

 Find the screen name of the account that posted the tweet you want to analyze. Place it right below. This will retrieve ths first two hundred off tweets that account. 


### Scrape tweets from multiple handles

In [ ]:
names = ["larrouturou"]  # placeholder screen name for the handle you want to retrieve tweets from
# YOU CAN QUERY MULTIPLE HANDLES, BUT REMEMBER TO HANDLE TWITTER'S TIMEOUT/LIMIT CONSTRAINTS

In [ ]:
""" Retrieves the most recent 200-ish tweets from a  list of given screen names
    Returns a data frame of the tweets with [id, screen_name, text, timestamp]
"""
def get_new_tweets(names):
  print("Retrieving tweets")
  corpus = []                                                                                        
  for name in names:
    tweets = api.user_timeline(name, include_rts=False, count=200, tweet_mode="extended")          
    time.sleep(4)
    corpus.extend(tweets)                                                                          
  data = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at] for tweet in corpus]
  tweets = pd.DataFrame(data, columns=['tweet_id', 'screen_name', 'text', 'timestamp'])      
            
  return tweets

In [ ]:
tweets = get_new_tweets (names)

Retrieving tweets


In [ ]:
tweets.head()

,tweet_id,screen_name,text,timestamp
0,1552978165756215296,larrouturou,Ce qui se passe “équivaut à des incendies sous...,2022-07-29 11:23:42
1,1548951688555397120,larrouturou,@mbrehin On le fait tous les jours! 💪 @Fresque...,2022-07-18 08:43:55
2,1548949157473357834,larrouturou,L'étude en question : https://t.co/itwSxY1BNN\...,2022-07-18 08:33:51
3,1548948519909728257,larrouturou,Projections de chercheurs français en 2017: ce...,2022-07-18 08:31:19
4,1548377536404393986,larrouturou,"L’été dernier, c’était les pluies torrentielle...",2022-07-16 18:42:26


### Scrape replies to a status

For definition of various Tweet objects see [here](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet).

In [51]:
def get_replies (user_name, tweet_id):
  replies = tweepy.Cursor(api.search, q='to:{}'.format(user_name),
                                since_id=tweet_id, tweet_mode='extended').items(5000)
  while True:
    try:
      reply = replies.next()
      if not hasattr(reply, 'in_reply_to_status_id_str'):
        continue
      if reply.in_reply_to_status_id==tweet_id:
        logging.info("reply of tweet:{}".format(reply.full_text))

    except tweepy.RateLimitError as e:
      logging.error("Twitter api rate limit reached".format(e))
      time.sleep(60)
      continue

    except tweepy.TweepError as e:
      logging.error("Tweepy error occured:{}".format(e))
      break

    except StopIteration:
      break

    except Exception as e:
      logger.error("Failed while fetching replies {}".format(e))
      break
  
  return reply

In [52]:
username ="larrouturou"
tweet_id = "1541498855061164034"

result = get_replies (user_name=username, tweet_id=tweet_id)

In [53]:
print (result)

Status(_api=<tweepy.api.API object at 0x7fa2d10cdb50>, _json={'created_at': 'Fri Jul 29 11:42:49 +0000 2022', 'id': 1552982976312627200, 'id_str': '1552982976312627200', 'full_text': '@larrouturou Titre trop équivoque, non? Espèces, méditerranée, remplacement?', 'truncated': False, 'display_text_range': [13, 76], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'larrouturou', 'name': 'Pierre Larrouturou', 'id': 18578222, 'id_str': '18578222', 'indices': [0, 12]}], 'urls': []}, 'metadata': {'iso_language_code': 'fr', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'in_reply_to_status_id': 1552978165756215296, 'in_reply_to_status_id_str': '1552978165756215296', 'in_reply_to_user_id': 18578222, 'in_reply_to_user_id_str': '18578222', 'in_reply_to_screen_name': 'larrouturou', 'user': {'id': 37893415, 'id_str': '37893415', 'name': 'Daniel ROGER🇺🇦', 'screen_name': 'DanPRAUDROGER', 'location':